In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
import pandas as pd

books = pd.read_csv(r"C:\Users\harip\Desktop\Semantic_Book_recommender\data\data_processed.csv")

In [13]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9.780002e+12,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883.0 A NOVEL THAT READERS and criti...
1,9.780002e+12,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982.0 A new 'Christie for Christmas'...
2,9.780006e+12,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736.0 A memorable, mesmerizing heroi..."
3,9.780006e+12,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897.0 Lewis' work on the nature of l...
4,9.780006e+12,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934.0 ""In The Problem of Pain, C.S. ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9.788172e+12,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222.0 On A Train Journey Home To Nor...
5193,9.788173e+12,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014.0 This book tells the tale of a ...
5194,9.788180e+12,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623.0 Wisdom to Create a Life of Pas...
5195,9.788185e+12,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535.0 This collection of the timeles...


In [ ]:
books["tagged_description"]


In [18]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False)

In [21]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1170, which is longer than the specified 100
Created a chunk of size 1216, which is longer than the specified 100


Created a chunk of size 375, which is longer than the specified 100
Created a chunk of size 311, which is longer than the specified 100
Created a chunk of size 485, which is longer than the specified 100
Created a chunk of size 484, which is longer than the specified 100
Created a chunk of size 962, which is longer than the specified 100
Created a chunk of size 190, which is longer than the specified 100
Created a chunk of size 845, which is longer than the specified 100
Created a chunk of size 298, which is longer than the specified 100
Created a chunk of size 199, which is longer than the specified 100
Created a chunk of size 883, which is longer than the specified 100
Created a chunk of size 1090, which is longer than the specified 100
Created a chunk of size 1191, which is longer than the specified 100
Created a chunk of size 306, which is longer than the specified 100
Created a chunk of size 272, which is longer than the specified 100
Created a chunk of size 213, which is longer t

In [22]:
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings())

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

In [ ]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]


In [ ]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [ ]:
retrieve_semantic_recommendations("A book to teach children about nature")
